In [1]:
import pandas as pd
import numpy as np
import torch
import os
import sys
from tqdm import tqdm, trange

sys.path.append("../../")
import biked_commons
from biked_commons.design_evaluation.design_evaluation import *
from biked_commons.data_loading import data_loading
from biked_commons.conditioning import conditioning
from biked_commons.design_evaluation.scoring import *

c:\Users\Lyler\mambaforge\envs\torch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")  

data = data_loading.load_bike_bench_train()

data_tens = torch.tensor(data.values, dtype=torch.float32, device=device)


In [3]:
evaluator, requirement_names, requirement_types = construct_tensor_evaluator(get_standard_evaluations(device), data.columns)

In [4]:
# num_data = data.shape[0]
num_data = 1
rider_condition = conditioning.sample_riders(num_data, split="test")
use_case_condition = conditioning.sample_use_case(num_data, split="test")
text_condition = conditioning.sample_text(num_data, split="test")
image_embeddings = conditioning.sample_image_embedding(num_data, split="test")
condition = {"Rider": rider_condition, "Use Case": use_case_condition, "Embedding": image_embeddings}
# condition = {"Rider": rider_condition, "Use Case": use_case_condition, "Text": text_condition}

In [5]:
#calcualte gradient of scores wrt data_tens

data_tens.requires_grad = True
eval_scores = evaluator(data_tens, condition)
score_sum = eval_scores.sum()
score_sum.backward()


In [6]:
#check for infs and nans in data_tens
if torch.any(torch.isnan(data_tens)) or torch.any(torch.isinf(data_tens)):
    print("Data tensor contains NaN or Inf values.")

In [7]:
#get nan indices of data_tens.grad
nan_indices = torch.isnan(data_tens.grad).nonzero(as_tuple=True)
print("nan indices: ", nan_indices)

nan indices:  (tensor([], device='cuda:0', dtype=torch.int64), tensor([], device='cuda:0', dtype=torch.int64))


In [8]:
isobjective = torch.tensor(requirement_types) == 1
isobjective = isobjective.to(device)
objective_scores = eval_scores[:, isobjective].detach().cpu().numpy()
# constraint_scores = eval_scores[:, ~isobjective].detach().numpy()

In [9]:
main_scorer = construct_scorer(MainScores, get_standard_evaluations(device), data.columns, device)
detailed_scorer = construct_scorer(DetailedScores, get_standard_evaluations(device), data.columns, device)

In [10]:
main_scorer(data_tens.detach(), condition)

Hypervolume                     0.251551
Constraint Satisfaction Rate    0.030000
Maximum Mean Discrepancy        0.000581
dtype: float64

In [11]:
detailed_scorer(data_tens.detach(), condition)

Min Objective Score: Usability Score - 0 to 1                                                                0.219893
Min Objective Score: Drag Force                                                                              8.979135
Min Objective Score: Knee Angle Error                                                                        0.000000
Min Objective Score: Hip Angle Error                                                                         9.419733
Min Objective Score: Arm Angle Error                                                                         0.000000
Min Objective Score: Cosine Distance to Embedding                                                            0.000178
Min Objective Score: Mass                                                                                    2.250693
Min Objective Score: Planar Compliance                                                                       0.000000
Min Objective Score: Transverse Compliance              